In [1]:
from torch.nn.functional import normalize
from GraphAwareNestedCVEvaluation import GraphAwareNestedCVEvaluation
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.utils import add_self_loops
from hyperopt import hp
import numpy as np
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
import shap
import torch
from NestedCV import index_to_mask

/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
cora_dataset = Planetoid(root='data/', name='Cora', split="public")
cora_dataset.transform = T.NormalizeFeatures()
cora_dataset[0].edge_index = add_self_loops(cora_dataset[0].edge_index)[0]

citeseer_dataset = Planetoid(root='data/', name='CiteSeer', split="public")
citeseer_dataset.transform = T.NormalizeFeatures()
citeseer_dataset[0].edge_index = add_self_loops(citeseer_dataset[0].edge_index)[0]

pubmed_dataset = Planetoid(root='data/', name='PubMed', split="public")
pubmed_dataset.transform = T.NormalizeFeatures()
pubmed_dataset[0].edge_index = add_self_loops(pubmed_dataset[0].edge_index)[0]

In [3]:
cora_dataset[0].x.to(torch.float16)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float16)

In [4]:
def norm_user_function(kwargs):
    return  normalize(kwargs["original_features"] + kwargs["summed_neighbors"], p=2.0, dim = 1)
    
def user_function(kwargs):
    return  kwargs["original_features"] + kwargs["summed_neighbors"]
    
class ModelSpace():
    def __init__(self):
        self.space = None
        self.initialize_space()

    def initialize_space(self):
        framework_choices = {
            'hops': [[0, 3, 8]],
            'attention_config': [None,{'inter_layer_normalize': False,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None}, 
                     {'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.01,
                     'dropout_attn': None},
                     {'inter_layer_normalize': True,
                     'use_pseudo_attention':True,
                     'cosine_eps':.001,
                     'dropout_attn': None}],
            'user_function': [norm_user_function, user_function],
        }
         
        self.space = {
            **{key: hp.choice(key, value) for key, value in framework_choices.items()}
        }
        
    def add_choice(self, key, items):
        self.space[key] = hp.choice(key, items)
        
    def add_uniform(self, key, limits: tuple):
        self.space[key] = hp.uniform(key, limits[0], limits[1])
        
    def add_loguniform(self, key, limits: tuple):
        self.space[key] = hp.loguniform(key, np.log(limits[0]), np.log(limits[1]))
        
    def add_qloguniform(self, key, limits, q):
        self.space[key] = hp.qloguniform(key, low=np.log(limits[0]), high=np.log(limits[1]), q=q)

class LogitsticRegressionSpace(ModelSpace):
    def __init__(self):
        super().__init__()

    def get_space(self):
        # self.add_choice('n_jobs', [-1])
        self.add_choice('class_weight', ["balanced"])
        self.add_loguniform('tol', [1e-4, 1e-2])
        # self.add_qloguniform('max_iter', [100, 20_000], 100)
        self.add_uniform('C', [0, 10]) ##TODO: 100?
        return self.space  

In [5]:
np.log(1e-2)

-4.605170185988091

In [6]:
lr_space = LogitsticRegressionSpace()

In [7]:
store = dict({})

In [ ]:
for dataset in [cora_dataset]: #[citeseer_dataset, pubmed_dataset, cora_dataset]
    data = dataset[0]
    graph_aware_nestedCV_evaluation = GraphAwareNestedCVEvaluation(2, LogisticRegression, data, max_evals= 100) # 2 instead of None
    graph_aware_nestedCV_evaluation.nested_cross_validate(3, 3, lr_space.get_space())
    store[dataset.name] = graph_aware_nestedCV_evaluation.nested_transd_cv

0it [00:00, ?it/s]

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/05 14:52:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/dwalke/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dwalke/git/graph_aware_ml/EnsembleFramework.py:328: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:231.)
  summed_exp_score = torch.zeros_like(exp_score).scatter(0, target,exp_score, reduce="add")
/home/dwalke/.local/lib/python3.10/site-packages/tr

In [ ]:
for item in store:
    print(f"""
    {item}:\n
    {store[item]}
    """)

In [ ]:
for dataset in store:
    graph_aware_nestedCV_evaluation = store[dataset]
    print(f"Required training time for {dataset}: {np.array(graph_aware_nestedCV_evaluation.train_times).mean():.1f} s.")

In [ ]:
store["Cora"]

In [ ]:
import matplotlib.pyplot as plt

class_colors = {
     0: '#1f77b4',  # Blue
    1: '#ff7f0e',  # Orange
    2: '#2ca02c',  # Green
    3: '#d62728',  # Red
    4: '#9467bd',  # Purple
    5: '#8c564b',  # Brown
    6: '#e377c2'   # Pink
}

for split_index, (train_index, test_index) in enumerate(store["Cora"].kf_outer.split(data.x, data.y)):
    train_mask = index_to_mask(data.x.shape[0], train_index)
    test_mask = index_to_mask(data.x.shape[0], test_index)
    background_datasets = store["Cora"].best_models[split_index].get_features(data.x, data.edge_index, train_mask, is_training=True)
    test_data = store["Cora"].best_models[split_index].get_features(data.x, data.edge_index, test_mask, is_training=False)
    shap_store = []
    fig =  plt.figure(figsize=(30, 15))
    gs = fig.add_gridspec(2, 3)
    
    for background_data_index, background_data in enumerate(background_datasets):
        explainer = shap.LinearExplainer(store["Cora"].best_models[split_index].trained_clfs[background_data_index], background_data.cpu().numpy())
        data_to_explain = test_data[background_data_index].cpu().numpy()
        shap_values_ex = explainer.shap_values(data_to_explain) #[0]
        shap_store.append(shap_values_ex)
        
        feature_names = [j for j in range(data_to_explain.shape[-1])]
        class_names = [i for i in range(7)]
        hop = store["Cora"].best_models[split_index].hops_list[background_data_index]

        ax = fig.add_subplot(gs[0, background_data_index])
        plt.sca(ax)
        shap.summary_plot(shap_values_ex, class_names= class_names, feature_names = feature_names, plot_type="bar", show=False, max_display=10, color = lambda i: class_colors[i],
                         class_inds='original')
        ax.set_xlabel("Mean |shap values|")
        ax.set_title(f"Neighborhood-order {hop}")
        ax.get_legend().remove()
        
    shap_values_avg = np.array(shap_store).mean(0)
    shap_values_avg = [shap_values_avg[i, :, :] for i in range(shap_values_avg.shape[0])]
    ax = fig.add_subplot(gs[1, :])
    plt.sca(ax)
    shap.summary_plot(shap_values_avg, plot_type="bar", class_names= class_names, feature_names = feature_names, max_display=20, color = lambda i: class_colors[i],
                         class_inds='original', show = False)
    ax.set_xlabel("Mean |shap values|")
    ax.set_title(f"GraphAware")
    ax.legend(fontsize=10, loc='lower right')
    plt.tight_layout()
    plt.show()